In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
from scipy import stats
#카이 제곱을 쓸수 있음
from ast import literal_eval
#파이썬 애플리케이션이 파이썬 추상 구문 문법 트리를 처리하는데 도움
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
#유사도, sklearn 머신러닝 도구를 사용
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import warnings; warnings.simplefilter('ignore')
#warning 제거

In [18]:
person = pd.read_csv('personality.csv', encoding='utf-8').dropna(how='all').drop('score_date',1)
movie = pd.read_csv('movies.csv',encoding='cp949')

#train_person, test_person = train_test_split(person, test_size=0.2,shuffle=False,random_state=1004)
#train_person

kf = KFold(n_splits=5)
for train_person, test_person in kf.split(person):
    
        train = person.iloc[train_person]
        test = person.iloc[test_person]
        train_user_id = train['user_id'].drop_duplicates( keep='first', inplace=False ).reset_index(drop=True)
        test_user_id = test['user_id'].drop_duplicates( keep='first', inplace=False ).reset_index(drop=True)
        movie_person = pd.merge(train, movie, on="movie_id", how="left")
        movie_person = movie_person.dropna(axis=0)
        
        movie_person_score = movie_person.drop(['movie_name', 'openness', 'neuroticism', 'conscientiousness', 'agreeableness', 'extraversion', 'movie_name'], axis=1)
        user_id_list = list(movie_person_score.groupby('user_id'))
        
        movie_person1 = movie_person.drop(['user_id', 'movie_id', 'movie_score', 'movie_name', 'openness', 'neuroticism', 'conscientiousness', 'agreeableness', 'extraversion', 'movie_name'], axis=1)
        x = movie_person1.values
        
        person1 = movie_person[['openness', 'neuroticism', 'conscientiousness', 'agreeableness', 'extraversion']]
        w = person1.values
        w = w.T

        w_personality = {'openness': np.round(w[0].reshape(len(w[0]) ,1), 3), 'agreeableness': np.round(w[1].reshape(len(w[1]) ,1), 3), 'neuroticism':np.round(w[2].reshape(len(w[2]) ,1), 3), 'conscientiousness': np.round(w[3].reshape(len(w[3]) ,1), 3), 'extraversion': np.round(w[4].reshape(len(w[4]) ,1), 3)}

        X_personality = {'openness': np.round(w_personality['openness']*x, 3), 'agreeableness':  np.round(w_personality['agreeableness']*x, 3), 'neuroticism':  np.round(w_personality['neuroticism']*x, 3), 'conscientiousness':  np.round(w_personality['conscientiousness']*x, 3), 'extraversion':  np.round(w_personality['extraversion']*x, 3)}

        
        
        learning_rate = 0.05
        num_epoch = 1000
        t_user = []

        for i in user_id_list:
            appended = False
            errors = []
            t_personality = np.full((15,1),1) # 초기 theta를 1로 초기화
            x = i[1].drop(['user_id', 'movie_id', 'movie_score'], axis=1) # x
            y = i[1]['movie_score'].values.reshape(len(i[1]),1) # y
            for epoch in range(num_epoch):
                y_hat = np.round(np.dot(x, t_personality), 2)
                loss =  y_hat - y
                cost = np.round(np.sum(np.round((loss**2) ,2) / len(x)), 2)
                gradient = np.round((np.dot(x.T, loss) / len(x)), 2)
                error = cost
        
                if len(errors) > 3:
                    if error < 0.1:
                        t_user.append((i[0], t_personality))
                        appended = True
                        break # error rate가 1 이하거나
            #elif error==errors[-1] and error==errors[-2] and error==errors[-3]:
               # t_user.append((i[0], t_personality))
                #break # error rate가 3번이상 똑같으면 break
                
                t_personality = t_personality - learning_rate * gradient
                errors.append(error)
                #if epoch % 50 == 0:
                    #print(epoch, " t = ", t, " error = ",error)
    
            if not appended :
                t_user.append((i[0], t_personality))
            #print("----" * 15)
            #print(i[0])
            #print(epoch ," t = ", t_personality, " error = ",error)
        #---------------- gradient end --------------------------
        
        Train_inputUser = train.drop('movie_id',1).drop('movie_score',1).drop_duplicates(keep='first', inplace=False)
        Test_inputUser = test.drop('movie_id',1).drop('movie_score',1).drop_duplicates(keep='first', inplace=False)
        Train_user_list = Train_inputUser.values.tolist()
        Test_user_list = Test_inputUser.values.tolist()
        
        
        
        
        
        Openness = []
        Neuroticism = []
        Conscientiousness = []
        Agreeableness = []
        Extraversion = []
        UserDistance = []

        for i in range(0,len(Test_user_list)):
            Distance = []
            for j in range(0,len(Train_user_list)):
                openness = (Test_user_list[i][1] - Train_user_list[j][1])
                neuroticism = (Test_user_list[i][2] - Train_user_list[j][2])
                conscientiousness = (Test_user_list[i][3] - Train_user_list[j][3])
                agreeableness = (Test_user_list[i][4] - Train_user_list[j][4])
                extraversion = (Test_user_list[i][5] - Train_user_list[j][5])
        
                Openness = openness**2
                Neuroticism = neuroticism**2
                Conscientiousness = conscientiousness**2
                Agreeableness = agreeableness**2
                Extraversion = extraversion**2
        
                Distance.append(Openness + Neuroticism + Conscientiousness + Agreeableness + Extraversion)
 
            for k in range(0,len(Distance)):
                if(Distance[k] == min(Distance)):
                    num_min = k
            UserDistance.append(num_min)
            
        #----------------- distance end ----------------
        ERROR = []
        for i in range(0, len(UserDistance)):
            for j in range(0, len(t_user)):
                if Train_user_list[UserDistance[i]][0] == t_user[j][0]:
                    x_test = user_id_list[j][1].drop(['user_id', 'movie_id', 'movie_score'], axis=1)
                    real_rating = user_id_list[j][1]['movie_score'].values.reshape(len(user_id_list[j][1]['movie_score'].values),1)
                    prediction_rating = np.dot(x_test, t_user[j][1])
                    error = abs(prediction_rating - real_rating) / real_rating * 100
            #print(error)
            #print(t_user[j][0])
            #print(sum(error)/len(error))
            ERROR.append(sum(error)/len(error))
        # ------------------- get error end -------------
        
        print(100 - sum(ERROR)/len(ERROR))

[72.49211598]
[71.02369153]
[71.66888455]
[71.91870593]
[71.27142096]
